In [1]:
import pandas as pd
import numpy as np
import random
import plotly as px
import plotly_express as go

import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored
import warnings

color_pal = sns.color_palette("husl", 9)
warnings.filterwarnings('ignore')


import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
df = pd.read_csv("AI modelling - DATA.csv")

# Assign new column names since they lack column names  
df.columns = ['Datetime', 'Temperature']
df

,Datetime,Temperature
0,2024-08-30 12:50:00,22.85
1,2024-08-30 12:50:00,22.85
2,2024-08-30 12:51:00,22.81
3,2024-08-30 12:51:00,22.84
4,2024-08-30 12:52:00,22.82
...,...,...
46979,2024-10-17 12:15:00,19.57
46980,2024-10-17 12:15:00,25.00
46981,2024-10-17 12:19:00,25.00
46982,2024-10-17 12:23:00,25.00


In [3]:
df.describe()

,Temperature
count,46984.000000
mean,21.529035
std,42.855390
min,3.120000
25%,18.450000
50%,20.810000
75%,22.060000
max,655.350000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46984 entries, 0 to 46983
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Datetime     46984 non-null  object 
 1   Temperature  46984 non-null  float64
dtypes: float64(1), object(1)
memory usage: 734.3+ KB


# CREATE MODEL 

In [5]:
from sklearn.model_selection import train_test_split

# Define your feature matrix (X) and target variable (y)
X = df.drop(columns=['Datetime'])
y = df['Temperature']

#prepare taining set
X_train, X_test, y_train, y_test = train_test_split( X ,y , random_state=1, test_size=0.20)


In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Define regression models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Support Vector Regressor': SVR(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'XGBoost Regressor': xgb.XGBRegressor()
}

# Train and evaluate models
for name, model in models.items():
    try:
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        print(f"{name}: {score}")
    except Exception as e:
        print(f"Error with {name}: {e}")

Linear Regression: 1.0
Decision Tree Regressor: 0.9999999988494087
Support Vector Regressor: 0.6690424305521728
K-Nearest Neighbors Regressor: 0.9999999976998778
Random Forest Regressor: 0.9999999989495822
XGBoost Regressor: 0.9999969118254671


In [7]:
from sklearn.linear_model import LinearRegression

# Create an SVR instance correctly
lf = LinearRegression()

# Fit the model
lf.fit(X_train, y_train)

# Print the score
print(lf.score(X_test, y_test))

# Predict on the test set
y_pred = lf.predict(X_test)

# Print predictions
print("Predictions of Average Temperatures Consumed:", y_pred)


1.0
Predictions of Average Temperatures Consumed: [ 4.46 23.62 23.97 ... 21.66 22.26 22.4 ]


# FEATURE ENGINEERING 

Creating New Features

In [8]:
# Convert the Datetime column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Create new columns
df['Year'] = df['Datetime'].dt.year
df['Month'] = df['Datetime'].dt.month
df['Day'] = df['Datetime'].dt.day
df['Hour'] = df['Datetime'].dt.hour

# Define time of day categories
def categorize_time_of_day(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Night'

df['TimeOfDay'] = df['Hour'].apply(categorize_time_of_day)

# Add day of the week (0 = Monday, 6 = Sunday)
df['Weekday'] = df['Datetime'].dt.weekday

# Let's set the data types to numeric
df['Year'] = df['Year'].astype(int)
df['Weekday'] = df['Weekday'].astype(int)


df.tail()


,Datetime,Temperature,Year,Month,Day,Hour,TimeOfDay,Weekday
46979,2024-10-17 12:15:00,19.57,2024,10,17,12,Afternoon,3
46980,2024-10-17 12:15:00,25.00,2024,10,17,12,Afternoon,3
46981,2024-10-17 12:19:00,25.00,2024,10,17,12,Afternoon,3
46982,2024-10-17 12:23:00,25.00,2024,10,17,12,Afternoon,3
46983,2024-10-17 12:25:00,19.62,2024,10,17,12,Afternoon,3


# SEASONALITY 

In [15]:
# Convert the 'datetime' column to datetime format
df = pd.to_datetime(df['Datetime'])



print(df)

KeyError: 'Datetime'

In [18]:
# Extract the day of the week
df = df.dt.day_name()

# Display the dataframe
print(df)

AttributeError: 'DataFrame' object has no attribute 'dt'

In [20]:
# Sample dataset with datetime column
df =  pd.read_csv("AI modelling - DATA.csv")

# Convert the 'datetime' column to pandas datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Extract the day of the week from the 'datetime' column
df['day_of_week'] = df['Datetime'].dt.day_name()

# Display the dataframe
print(df)

KeyError: 'Datetime'

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("AI modelling - DATA.csv")

# Check the column names
print(df.columns)

# Convert the 'Datetime' column to pandas datetime format (adjust the column name if necessary)
df['Datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')  # Use 'errors=coerce' to handle invalid formats

# Check for any rows where the conversion failed
print(df[df['Datetime'].isna()])

# Extract the day of the week if the conversion was successful
df['day_of_week'] = df['Datetime'].dt.day_name()

# Display the updated dataframe
print(df)
